In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

Matplotlib is building the font cache; this may take a moment.


In [2]:
file_path = "/Users/kilian_1/Desktop/Education/Bayes_MSc_Energy_Trade_and_Finance/Term_3/Market Microstructure/Market-Microstucture-Rep/SMM921_pf_data_2025.xlsx"

xls = pd.ExcelFile(file_path)

# List all sheet names
sheet_names = xls.sheet_names
print(sheet_names)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kilian_1/Desktop/Education/Bayes_MSc_Energy_Trade_and_Finance/Term_3/Market Microstructure/Market-Microstucture-Rep/SMM921_pf_data_2025.xlsx'

In [ ]:
#Sheet No 1
df = xls.parse('Sheet1')
# Display the first few rows of the DataFrame
print(df.head())

In [ ]:
#Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

returns_df = df.iloc[:, 1:].pct_change() # Uses iloc (integer-location based indexing) to select all rows (:) and all columns starting from the second one (1:).

returns_df.insert(0, 'Date', df['Date'])  # Insert the 'Date' column back into the returns DataFrame

#Visualise the dataframe
print(returns_df.head())


desc = returns_df.describe()

# 5. Prepare a text table with 2‐decimal formatting
cell_text = [
    [f"{val:.2f}" for val in desc.loc[row].values]
    for row in desc.index
]

# 6. Plot the table
fig, ax = plt.subplots(figsize=(12, 3))
ax.axis('off')

table = ax.table(
    cellText=cell_text,
    rowLabels=desc.index,
    colLabels=desc.columns,
    cellLoc='center',
    rowLoc='center',
    loc='center'
)

# 7. Adjust overall font size
table.auto_set_font_size(False)
table.set_fontsize(6)
table.scale(1, 1.2)

# 8. Increase header font size and weight
for (row, col), cell in table.get_celld().items():
    if row == 0 or col == -1:
        cell.set_text_props(fontsize=4.5, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
returns_df["world stock market return"] = returns_df.iloc[:, 1:].mean(axis = 1) #axis = 1 means we are calculating the mean across columns for each row
print(returns_df.head())

In [ ]:
returns_clean = returns_df.dropna()  # Drop rows with NaN values
# Q1: Full-sample annualised return statistics per country


# Reshape the data: long format (Date, Country, Monthly Return)
returns_formatted = returns_clean.melt(id_vars=["Date"], var_name="Country", value_name="Monthly Return")

# Group by Country and compute full-period annualised stats
annual_stats = returns_formatted.groupby("Country").agg(
    annualised_returns=("Monthly Return", lambda x: x.mean() * 12),
    annualised_std=("Monthly Return", lambda x: x.std() * np.sqrt(12)),
    annualised_sharpe=("Monthly Return", lambda x: (x.mean() / x.std()) * np.sqrt(12))
).reset_index()

# Print results
print(annual_stats.head(15))


In [ ]:
# Sort the summary by Sharpe ratio for clearer visualization
annual_stats_sorted = annual_stats.sort_values("annualised_sharpe", ascending=False)

# Plot all 3 metrics in one figure
fig, axs = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Bar chart: Annualised Mean Returns
axs[0].bar(annual_stats_sorted["Country"], annual_stats_sorted["annualised_returns"], color="skyblue", edgecolor="black")
axs[0].set_title("Annualised Mean Returns by Country")
axs[0].set_ylabel("Return")
axs[0].grid(axis="y", linestyle="--", alpha=0.7)

# Bar chart: Annualised Standard Deviation
axs[1].bar(annual_stats_sorted["Country"], annual_stats_sorted["annualised_std"], color="orange", edgecolor="black")
axs[1].set_title("Annualised Volatility (Std Dev) by Country")
axs[1].set_ylabel("Volatility")
axs[1].grid(axis="y", linestyle="--", alpha=0.7)

# Bar chart: Sharpe Ratio
axs[2].bar(annual_stats_sorted["Country"], annual_stats_sorted["annualised_sharpe"], color="green", edgecolor="black")
axs[2].set_title("Annualised Sharpe Ratio by Country")
axs[2].set_ylabel("Sharpe Ratio")
axs[2].grid(axis="y", linestyle="--", alpha=0.7)

# Formatting
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
#Q2
'''Create, for each of the countries, the beta against the world return series. Plot them,
interpret your findings (in terms of risk exposures for countries) and identify any
outliers. [Note: there is no need to account for the risk-free rate when computing the
betas.]'''

regression_data = returns_df.dropna()


#Compute explanatory variable + add constant (Beta_0)
X = sm.add_constant(regression_data["world stock market return"])

#dictionnary to stoe the betas
betas = {}

for country in regression_data.columns:
    if country != "Date" and country != "world stock market return":
        # Dependent variable (returns of the country)
        y = regression_data[country]
        
        # Fit the OLS regression model
        model = sm.OLS(y, X).fit()
        
        # Store the beta coefficient (slope) in the dictionary
        betas[country] = model.params[1]  # The beta is the second parameter (index 1)

# Convert to df
betas_df =  pd.DataFrame.from_dict(betas, orient= "index", columns= ["Betas"])
betas_df = betas_df.sort_values("Betas", ascending= False)

plt.figure(figsize=(14, 6))
betas_df["Betas"].plot(kind='bar', color='blue', edgecolor='black')
plt.title("Country Betas Against World Market Return")
plt.ylabel("Beta")
plt.xlabel("Country")
plt.xticks(rotation=45, ha='right')
plt.axhline(1.0, color='gray', linestyle='--', linewidth=1, label='Beta = 1')
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
#Q3
momentum_signals = pd.DataFrame(index=returns_df.index)
#For each country and month: Momentum signal = sum of returns over months t–12 to t–2 ( Exclude the most recent month (t–1))
for country in returns_df.columns:
    if country not in ["Date", "world stock market return"]:   
        momentum_signals[country] = returns_df[country].shift(2).rolling(window=11).sum()

print(momentum_signals.head(15))

In [ ]:
def create_portfolio_returns():
    portfolio_returns = {}
    for i in range(1, 6):
        portfolio_returns[i] = []
    return portfolio_returns

# Usage
portfolio_returns = create_portfolio_returns()
print(portfolio_returns)
dates = []


for t in range(len(momentum_signals) - 1):  # up to second last row
    current_date = momentum_signals.index[t]
    next_date = momentum_signals.index[t + 1]

    current_mom = momentum_signals.iloc[t]
    next_returns = returns_df.iloc[t + 1]

    valid_mom = current_mom.dropna()
    valid_mom = valid_mom[valid_mom.index != "Date"]
    valid_mom = valid_mom[valid_mom.index != "world stock market return"]
    

    if len(valid_mom) >= 5: #Only proceed if we have at least 5 valid momentum values (i.e., assets). This ensures each group/quintile has at least 1 asset.
        ranked = valid_mom.sort_values() #Sort assets by their momentum scores from lowest to highest.
        groups = np.array_split(ranked.index, 5) #Split the ranked list  into 5 equal-sized groups → this forms the 5 portfolios (quintiles).
        
        #enumerate(groups, 1) starts counting from 1.
        for i, group in enumerate(groups, 1): #Loop over the 5 groups (group is a list of asset names like ['USA', 'UK']) 
            mean_return = next_returns[group].mean() #next_returns[group] fetches the actual returns at time t+1 for those assets
            portfolio_returns[i].append(mean_return) #.mean() calculates the average return of that group

        dates.append(next_date) #portfolio_returns[i].append(...) stores the result in a dictionary like:

#----------------------------------------------------------------------------------------------------------
# Step 4: Create DataFrame of monthly returns for each momentum portfolio
momentum_portfolios = pd.DataFrame(portfolio_returns, index=dates)
print(momentum_portfolios)
momentum_portfolios.columns = [f"Portfolio {i}" for i in momentum_portfolios.columns]

# Step 5: Compute annualised stats
mean_returns = momentum_portfolios.mean() * 12
std_returns = momentum_portfolios.std() * (12 ** 0.5)
sharpe_ratios = mean_returns / std_returns

summary_stats = pd.DataFrame({
    "Annualised Mean Return": mean_returns,
    "Annualised Std Dev": std_returns,
    "Sharpe Ratio": sharpe_ratios
})

# Step 6: Plot cumulative returns
cumulative_returns = (1 + momentum_portfolios).cumprod()
print(cumulative_returns)


import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
for col in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[col], label=col)
plt.title("Cumulative Returns of Momentum Portfolios")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 7: Build High-minus-Low (HML) portfolio
hml_returns = momentum_portfolios["Portfolio 5"] - momentum_portfolios["Portfolio 1"]
hml_cumulative = (1 + hml_returns).cumprod()
print(hml_cumulative)

# Step 8: HML summary stats
hml_mean = hml_returns.mean() * 12
hml_std = hml_returns.std() * (12 ** 0.5)
hml_sharpe = hml_mean / hml_std

# Step 9: Plot HML cumulative return
plt.figure(figsize=(10, 5))
plt.plot(hml_cumulative.index, hml_cumulative, label="HML Portfolio", color="black")
plt.title("Cumulative Return of HML (High-Minus-Low) Portfolio")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Step 10: Regress HML on World Market Return
import statsmodels.api as sm

world_returns_aligned = returns_df.loc[hml_returns.index, "world stock market return"] # hml_returns.index It represents the time points for which the High-Minus-Low (HML) portfolio returns were computed.
X = sm.add_constant(world_returns_aligned)
model = sm.OLS(hml_returns, X).fit()
hml_beta = model.params["world stock market return"]
hml_alpha = model.params["const"]



hml_summary = pd.DataFrame({
    "HML Mean Return": [hml_mean],
    "HML Std Dev": [hml_std],
    "HML Sharpe Ratio": [hml_sharpe],
    "HML Alpha": [hml_alpha],
    "HML Beta to World": [hml_beta]
})
print(hml_summary)





In [ ]:
total_countries = 35
total_months = 240
start_opti_month = 60  # index for 61st month


countries = []
for i in range(total_countries):
    countries.append(f"Country_{i+1}")
print(countries)

dates = pd.date_range(start="2005-01-01", periods=total_months, freq="ME")


momentum_signals = momentum_signals.dropna()

returns_df = returns_df.loc[momentum_signals.index]

#Param
IC = 0.02
risk_aversion = 4
start_month = 60 #Range starts from 0, so 60 instead of 61

opt_returns = []
opt_weights_list = []
turnovers = []
dates = momentum_signals.index[start_month:]

# Optimization loop
for t in range(start_month, len(momentum_signals) - 1): #60 to len -1
    current_date = momentum_signals.index[t]
    next_date = momentum_signals.index[t + 1]

    

    













